<a href="https://colab.research.google.com/github/antoine123456/Monte-Cuda/blob/main/virginCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-utzd66l7
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-utzd66l7
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=183b6737dd8dacf3bd5b31583d706cb44bfaf410e0e8ae353aec2db433bba11e
  Stored in directory: /tmp/pip-ephem-wheel-cache-c19bx4u8/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name testGoogleColab.cu

#include <stdio.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char* file, int line, bool abort = true)
{
    if (code != cudaSuccess)
    {
        fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
        if (abort) exit(code);
    }
}

__global__ void add(int a, int b, int *c) {
   *c = a + b; }

int main() {

   // --- Host declarations and initializations
   int a, b, c;
   a = 2;
   b = 6;

   // --- Device allocations
   int *d_c; gpuErrchk(cudaMalloc(&d_c, sizeof(int)));

   // --- Kernel execution
   add<<<1,1>>>(a, b, d_c);
   gpuErrchk(cudaPeekAtLastError());
   gpuErrchk(cudaDeviceSynchronize());

   // --- Moving the results from device to host
   gpuErrchk(cudaMemcpy(&c, d_c, sizeof(int), cudaMemcpyDeviceToHost));

   // --- Results printout
   printf("%d + %d is %d\n", a, b, c);
   
   return 0; }

'File written in /content/src/testGoogleColab.cu'

In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp38-cp38-linux_x86_64.whl size=646530 sha256=3e20c46a8b5e987cafc502c85c41850a3d96234668731ffcee1c921c84153a4a
  Stored in directory: /root/.cache/pip/wheels/7b/41/0d/7cecb04af969d283ebe4a69579a8b2baec0d010a1ac4159f7e
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=14611ad935d017d55dc097acc842954e6606b98a64412ba5c3b33a31b7363f1d
  Stored in directory: /root/.cache/pip/wheels/cb/fc/

In [ ]:
import pycuda.driver as drv
import pycuda.autoinit
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


In [ ]:
%%cuda --name piCuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <time.h>
__global__ void calculate_pi(curandState state, int *count, int size)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    curand_init(0, i, 0, state);
    if (i >= size)
        return;
    float x = curand_uniform(&state[i]);
    float y = curand_uniform(&state[i]);
    count[i] = 0;
    if (x * x + y * y < 1)
        count[i] = 1;
}

int main()
{
    int size = 9999;

    int *count;
    cudaMalloc(&count, size*sizeof(int));

    int *count_d;
    cudaMalloc(&count_d, size*sizeof(int));

    int blockSize = 256;
    int numBlocks = (size + blockSize - 1) / blockSize;

    curandState state;
    calculate_pi<<<numBlocks, blockSize>>>(state, count_d, size);

    cudaMemcpy(&count, count_d, sizeof(int), cudaMemcpyDeviceToHost);
    // double pi = 4.0 * count / size;
    // printf("%f", pi);
    printf("%d", *count);

    cudaFree(count);
    return 0;
}

'File written in /content/src/piCuda.cu'

In [ ]:
!nvcc -arch=sm_75 -o "/content/src/testGoogleColab.o" /content/src/testGoogleColab.cu

In [ ]:
!chmod 755 /content/src/testGoogleColab.o
!/content/src/testGoogleColab.o

2 + 6 is 8


In [ ]:
!nvcc -arch=sm_75 -o "/content/src/piCuda.o" /content/src/piCuda.cu

In [ ]:
!chmod 755 /content/src/piCuda.o
!/content/src/piCuda.o

3.175918

3.12